In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
                    .appName("Analyzing soccer players")\
                    .getOrCreate()

In [3]:
players = spark.read\
            .format("csv")\
            .option("header", "true")\
            .load("/data/pluralsight_spark2/02/demos/datasets/player.csv")

In [4]:
players.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)



In [5]:
players.show(5)

+---+-------------+------------------+------------------+-------------------+------+------+
| id|player_api_id|       player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+------------------+------------------+-------------------+------+------+
|  1|       505942|Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|   Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|       Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|     Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|      Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
+---+-------------+------------------+------------------+-------------------+------+------+
only showing top 5 rows



In [6]:
player_attributes = spark.read\
            .format("csv")\
            .option("header", "true")\
            .load("/data/pluralsight_spark2/02/demos/datasets/player_attributes.csv")

In [7]:
player_attributes.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- potential: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- attacking_work_rate: string (nullable = true)
 |-- defensive_work_rate: string (nullable = true)
 |-- crossing: string (nullable = true)
 |-- finishing: string (nullable = true)
 |-- heading_accuracy: string (nullable = true)
 |-- short_passing: string (nullable = true)
 |-- volleys: string (nullable = true)
 |-- dribbling: string (nullable = true)
 |-- curve: string (nullable = true)
 |-- free_kick_accuracy: string (nullable = true)
 |-- long_passing: string (nullable = true)
 |-- ball_control: string (nullable = true)
 |-- acceleration: string (nullable = true)
 |-- sprint_speed: string (nullable = true)
 |-- agility: string (nullable = true)
 |-- reactions: string (nullable = true

In [8]:
players.count(), player_attributes.count()

(11060, 183978)

In [9]:
player_attributes.select('player_api_id')\
                    .distinct()\
                    .count()

11060

In [10]:
players = players.drop('id', 'player_fifa_api_id')

In [11]:
players.columns

['player_api_id', 'player_name', 'birthday', 'height', 'weight']

In [12]:
player_attributes = player_attributes.drop(
    'id',
    'player_fifa_api_id',
    'preffered_foot',
    'attacking_work_rate',
    'defensive_work_rate',
    'crossing',
    'jumping',
    'sprint_speed',
    'balance',
    'aggression',
    'short_passing',
    'potential'
)
player_attributes.columns

['player_api_id',
 'date',
 'overall_rating',
 'preferred_foot',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [13]:
player_attributes = player_attributes.dropna()
players = players.dropna()

In [14]:
players.count(), player_attributes.count()

(11060, 181265)

In [18]:
from pyspark.sql.functions import udf

In [21]:
year_extract_udf = udf(lambda date: date.split('-')[0])

In [22]:
player_attributes = player_attributes.withColumn(
    "year", 
    year_extract_udf(player_attributes.date)
)

In [23]:
player_attributes.show(5)

+-------------+-------------------+--------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-------------+-----------+------+---------+-------+---------------+--------------+---------+-----------+----------+--------------+-----------+----+
|player_api_id|               date|overall_rating|preferred_foot|finishing|heading_accuracy|volleys|dribbling|curve|free_kick_accuracy|long_passing|ball_control|acceleration|agility|reactions|shot_power|stamina|strength|long_shots|interceptions|positioning|vision|penalties|marking|standing_tackle|sliding_tackle|gk_diving|gk_handling|gk_kicking|gk_positioning|gk_reflexes|year|
+-------------+-------------------+--------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-----------

In [24]:
player_attributes.drop('date')

DataFrame[player_api_id: string, overall_rating: string, preferred_foot: string, finishing: string, heading_accuracy: string, volleys: string, dribbling: string, curve: string, free_kick_accuracy: string, long_passing: string, ball_control: string, acceleration: string, agility: string, reactions: string, shot_power: string, stamina: string, strength: string, long_shots: string, interceptions: string, positioning: string, vision: string, penalties: string, marking: string, standing_tackle: string, sliding_tackle: string, gk_diving: string, gk_handling: string, gk_kicking: string, gk_positioning: string, gk_reflexes: string, year: string]

In [26]:
player_attributes.columns

['player_api_id',
 'date',
 'overall_rating',
 'preferred_foot',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'year']

In [27]:
player_attributes.show(5)

+-------------+-------------------+--------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-------------+-----------+------+---------+-------+---------------+--------------+---------+-----------+----------+--------------+-----------+----+
|player_api_id|               date|overall_rating|preferred_foot|finishing|heading_accuracy|volleys|dribbling|curve|free_kick_accuracy|long_passing|ball_control|acceleration|agility|reactions|shot_power|stamina|strength|long_shots|interceptions|positioning|vision|penalties|marking|standing_tackle|sliding_tackle|gk_diving|gk_handling|gk_kicking|gk_positioning|gk_reflexes|year|
+-------------+-------------------+--------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-----------

In [28]:
pa_2016 = player_attributes.filter(player_attributes.year == 2016)

In [30]:
pa_2016.count()

14098

In [34]:
pa_2016.select(pa_2016.player_api_id)\
.distinct()\
.count()


5586

In [35]:
pa_striker_2016 = pa_2016.groupBy('player_api_id')\
.agg({
    "finishing":"avg",
    "shot_power":"avg",
    "acceleration":"avg"
})

In [36]:
pa_striker_2016.count()

5586

In [37]:
pa_striker_2016.show(5)

+-------------+-----------------+-----------------+---------------+
|player_api_id|   avg(finishing)|avg(acceleration)|avg(shot_power)|
+-------------+-----------------+-----------------+---------------+
|       309726|75.44444444444444|74.11111111111111|           76.0|
|        26112|             53.0|             51.0|           76.0|
|        38433|            68.25|             74.0|           74.0|
|       295060|             25.0|             62.0|           40.0|
|       161396|             29.0|             72.0|           69.0|
+-------------+-----------------+-----------------+---------------+
only showing top 5 rows



In [39]:
pa_striker_2016 = pa_striker_2016.withColumnRenamed("avg(finishing)", "finishing")\
.withColumnRenamed("avg(shot_power)", "shot_power")\
.withColumnRenamed("avg(acceleration)", "acceleration")

In [40]:
weight_finishing = 1
weight_shot_power = 2
weight_acceleration = 1
total_weight = weight_finishing + weight_shot_power + weight_acceleration